In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/cedric/Datasets/semantic_kitti/')
SEQUENCE_NUM = 7

ind_start = 0
ind_end = 100
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = True 

out_folder_ncuts = 'test_data/'

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)




out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)

In [ ]:
process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                              major_voxel_size=major_voxel_size,icp=True,
                              out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                              ground_segmentation_method=ground_segmentation_method)


In [ ]:
##load data if already stored 
pcd_ground_minor, pcd_nonground_minor,\
	all_poses, T_pcd, first_position = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                     ground_mode=ground_segmentation_method)

Now we subsample the poses based on a voxel_size

In [ ]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size)

In [ ]:
alpha = 0.0
theta = 0.0
beta = 1.0
gamma = 0.0
proximity_threshold = 1.0

patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
for sequence in range(len(center_ids)):
        if NCUT_ground == False : 
                merged_chunk,file_name = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices)
                
                o3d.io.write_point_cloud(file_name, merged_chunk, write_ascii=False, compressed=False, print_progress=False)

                
        else : 
                obstacle_out,file_name = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)
                
                
                ground_out,file_name = ncuts_chunk(dataset,indices_ground,pcd_ground_chunks,None,
                        pcd_ground_chunks_major_downsampling,
                        pcd_ground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)

                o3d.io.write_point_cloud(file_name, obstacle_out + ground_out, write_ascii=False, compressed=False, print_progress=False)

                print("Pointcloud written to file")

        
        
        

In [ ]:
point_clouds = get_merge_pcds(out_folder_ncuts)
merge = merge_chunks_unite_instances(point_clouds)
o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)

In [ ]:
##load merge pcd 
merge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')

In [ ]:
DATASET_PATH = os.path.join('/Users/cedric/Datasets/semantic_kitti/')
SEQUENCE_NUM = 7

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=False)


In [ ]:
from open3d.pipelines import registration
import numpy as np 
merge.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))

for i in range(70,72):
	local_pcd = o3d.geometry.PointCloud()
	local_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	
	
	## label visualization 
	labeled_pcd = o3d.geometry.PointCloud()
	labeled_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	labeled_pcd.colors = o3d.utility.Vector3dVector(np.vstack([0,0,0] for i in range(np.asarray(labeled_pcd.points).shape[0])))
	panoptic_labels = dataset[i].panoptic_labels # semantics + panoptics combined 
	semantic_labels = dataset[i].semantic_labels
	instance_labels = dataset[i].instance_labels
	intensity = dataset[i].intensity
	
	transform = dataset.get_pose(i)
	local_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))
	reg_p2l = registration.registration_icp(local_pcd, merge, 0.9, transform, registration.TransformationEstimationPointToPlane(), registration.ICPConvergenceCriteria(max_iteration=1000))
	transform = reg_p2l.transformation
	local_pcd.transform(transform)
	
	local_pcd.normals = o3d.utility.Vector3dVector([])
	
	local_pcd.paint_uniform_color([0, 0, 0])
	
	
	
	
	colors, labels,local_labels = kDTree_1NN_feature_reprojection_colors(np.asarray(local_pcd.colors), local_pcd, np.asarray(merge.colors), merge,panoptic_labels, max_radius=0.2)
	labeled_pcd = color_pcd_by_labels(labeled_pcd,labels.reshape(-1,))
	local_pcd.colors = o3d.utility.Vector3dVector(colors)
	#o3d.io.write_point_cloud("test.pcd", local_pcd, write_ascii=False, compressed=False, print_progress=False)
	
	
	
	labeled_pcd.translate([0,120,0])
	o3d.visualization.draw_geometries([local_pcd,labeled_pcd])
	unique_colors, labels = np.unique(colors, axis=0, return_inverse=True)
	unseen_mask = np.all(colors == [1,0,0], axis=1)
	labels[unseen_mask] = -1
	street_mask = np.all(colors == [0,0,0], axis=1)
	labels[street_mask] = 1
	
	#np.savez('output_files/7_tarl/ ' + str(i) + '.npz',labels=labels)

	

	#np.savez('output_files/7_original/' + str(i) + '.npz',ncut_labels=local_labels,kitti_labels=labels,pts=np.asarray(local_pcd.points),           
	#intensities=intensity,kitti_labels_semantic=semantic_labels,kitti_labels_instance=instance_labels)


In [ ]:
import numpy as np 
with np.load('output_files/7_original/0.npz') as data:
            xyz = data['pts'].astype(np.float)
            labels = data['ncut_labels'].astype(np.int32)  
            kitti_labels = data['kitti_labels']
            intensity = data['intensities']
            
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd = color_pcd_by_labels(pcd,labels)
o3d.visualization.draw_geometries([pcd])
        